In [1]:
#Exercise 4:

#subset size = 50000:

import pandas as pd;
from scipy.stats import zscore
import torch as torch;

import numpy as np

import torchvision.datasets as datasets


np.random.seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

n_classes = 10;
n_features = 28*28;


In [2]:
full_train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=None)
full_test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=None)

x_train = full_train_dataset.data.numpy().reshape(-1,n_features).astype(dtype=np.float64)/255.0;
x_test = full_test_dataset.data.numpy().reshape(-1,n_features).astype(dtype=np.float64)/255.0;

y_train_cat = full_train_dataset.targets.numpy()
y_test_cat = full_test_dataset.targets.numpy()

In [3]:
dd_y=pd.DataFrame(data=y_train_cat)
y_train=pd.get_dummies(dd_y,columns = list(dd_y.columns)).to_numpy().astype(dtype=float);

dd_y=pd.DataFrame(data=y_test_cat)
y_test=pd.get_dummies(dd_y,columns = list(dd_y.columns)).to_numpy().astype(dtype=float);


# permute the dataset
permutation = np.random.permutation(x_train.shape[0]);
x_train=x_train[permutation,:]
y_train=y_train[permutation,:]



In [4]:
subset_size = 50000;
x_train=x_train[:subset_size,:]
y_train=y_train[:subset_size,:]


n_train=x_train.shape[0];

# create tensor variables for data, we do not need gradient w.r.t. to them
t_x_test=torch.tensor(x_test,requires_grad=False,device=device);
t_y_test=torch.tensor(y_test,requires_grad=False,device=device);


In [6]:
#define starting value of weights W for gradient descent
init_std_dev = 0.01;
initialW=init_std_dev*np.random.randn(n_features,n_classes)
# create a PyTorch tensor variable for W and b. 
# we will be optimizing over W, b, finding their best value using gradient descent,
#so we need gradient enabled
W = torch.tensor(initialW,requires_grad=True,device=device);
b = torch.zeros((1,n_classes),requires_grad=True,device=device);
# this optimizer will do gradient descent for us
learning_rate = 0.01;
#optimizer = torch.optim.SGD([W,b],lr=learning_rate)
optimizer = torch.optim.Adam([W,b],lr=learning_rate)
n_epochs = 10000;
batch_size = 64;

In [7]:
for i in range(n_epochs):

    
    permutation = np.random.permutation(x_train.shape[0]);
    x_train=x_train[permutation,:]
    y_train=y_train[permutation,:]

    accuracy = 0.0;

    
    for j in range(0,n_train,batch_size):
        t_x_train = torch.tensor(x_train[j:j+batch_size,:],requires_grad=False,device=device);
        t_y_train = torch.tensor(y_train[j:j+batch_size,:],requires_grad=False,device=device);

        # clear previous gradient calculations
        optimizer.zero_grad();
        
        # calculate model predictions
        linear_predictions = torch.matmul(t_x_train,W)+b
        activations =  torch.exp(linear_predictions);
    
        #Passing through softmax
        sum_activations = torch.sum(activations,dim=1,keepdim=True)
        normalized_activations = torch.div(activations, sum_activations);
        cross_entropy_loss = torch.sum(torch.multiply(t_y_train,torch.log(normalized_activations)),dim=1 )
        risk = -1.0 * torch.mean(cross_entropy_loss );

        # calculate gradients
        risk.backward();
        
        # take the gradient step
        optimizer.step();
        

        batch_risk=risk.item();

        true_class = np.argmax(t_y_train.detach().cpu().numpy(),axis=1)
        pred_class = np.argmax(normalized_activations.detach().cpu().numpy(),axis=1)
        accuracy += np.count_nonzero(true_class == pred_class);
        
    
    #  turn off gradients
        
    with (torch.no_grad()):
      linear_predictions = torch.matmul(t_x_test,W)+b
      activations =  torch.exp(linear_predictions);
    
      #Passing through softmax for test risk and accuracy
      sum_activations = torch.sum(activations,dim=1,keepdim=True)
      test_normalized_activations = torch.div(activations, sum_activations);
      cross_entropy_loss = torch.sum(torch.multiply(t_y_test,torch.log(test_normalized_activations)),dim=1 )
    
      test_risk = -1.0 * torch.mean(cross_entropy_loss );
    
      test_true_class = np.argmax(t_y_test.detach().cpu().numpy(),axis=1)
      test_pred_class = np.argmax(test_normalized_activations.detach().cpu().numpy(),axis=1)
      test_accuracy = np.count_nonzero(test_true_class == test_pred_class)/test_pred_class.shape[0];
      test_error = 1.0 - test_accuracy;
        
    accuracy = accuracy / float(x_train.shape[0])
    print(i,batch_risk,test_risk.item(),accuracy,test_accuracy)

0 0.20120659800212945 0.34437799544674064 0.89842 0.9039
1 0.44320228364757597 0.32755987978589785 0.91504 0.9096
2 0.21697514300489432 0.32124663176711016 0.91816 0.9113
3 0.11906762774954913 0.3125621131051566 0.91888 0.9166
4 0.11129720199532905 0.3220847921226752 0.91984 0.9141
5 0.19424294318263804 0.3199239374966278 0.9205 0.9205
6 0.04973977145779209 0.34800477815973824 0.92136 0.9126
7 0.19045818364888975 0.35930432079602753 0.92286 0.9076
8 0.9049256373296897 0.3374018339422252 0.92236 0.9166
9 0.00789742783402042 0.3348684898716689 0.92252 0.9192
10 0.09876147668191611 0.3802268827443158 0.9248 0.9037
11 0.4111200990865403 0.3426976953624528 0.92482 0.9215
12 0.2988055907337234 0.3454208595111184 0.92348 0.9177
13 0.08768330408741794 0.3587999845187688 0.92404 0.9176
14 0.10261616152802921 0.3841646347857667 0.92468 0.9088
15 0.14043554115971216 0.3651800331612018 0.92506 0.9169
16 0.07584852842787634 0.37861391390975774 0.92636 0.9056
17 0.25242392477047115 0.386671896926597

145 0.18087621145686975 0.5259339520788019 0.92934 0.9127
146 0.12538266997172057 0.5480836744597999 0.9299 0.9091
147 0.14486748836415103 0.5420051580102914 0.92926 0.9066
148 0.06591172976646563 0.5449416558002581 0.92958 0.9071
149 0.18897114911785395 0.5552549588910408 0.92948 0.9092
150 0.31107439429459016 0.551796818192727 0.92886 0.9054
151 0.9629924963356338 0.5359321726415286 0.9299 0.9106
152 0.1699823967954159 0.5325433336438724 0.9291 0.9144
153 0.6598728537268863 0.5330966062074127 0.92972 0.9098
154 0.1251603392384683 0.5316743662416952 0.92974 0.9131
155 0.1700008071528275 0.5510312817685522 0.92838 0.9063
156 0.01359086681663026 0.535793321028996 0.92858 0.9147
157 0.05802855633318932 0.5632308672133051 0.92914 0.9063
158 0.34805091425482 0.5441258070899138 0.92906 0.9104
159 0.008988975374010057 0.5403102980071776 0.92934 0.9091
160 0.5381073489433638 0.5443156030131328 0.9298 0.9091
161 0.06778849152887417 0.5518647969383799 0.92864 0.907
162 0.11872068304592018 0.556

289 0.060228929928253766 0.6274676739455467 0.93006 0.9107
290 0.33031690313586715 0.6242311798446998 0.9291 0.9091
291 0.24041015466023496 0.6452129570443722 0.92946 0.9043
292 0.21193059653278634 0.6311563154738095 0.93006 0.9086
293 0.03527088597558451 0.6148298844538527 0.92934 0.9095
294 0.05198091541390521 0.6254038095050237 0.93012 0.9084
295 0.45825487141240173 0.6337344542904734 0.92996 0.9096
296 0.06839835386362493 0.6438840135239616 0.92844 0.904
297 0.11303595338460909 0.6291357069919836 0.92974 0.9064
298 0.06726825680981724 0.653389898259438 0.9307 0.9031
299 0.21035081780961024 0.6391615456814189 0.92842 0.9086
300 0.1419260530106282 0.6179935219823391 0.93068 0.9111
301 0.04360437518944582 0.623130388061755 0.92918 0.9107
302 0.4117677021460345 0.6463083678666053 0.9301 0.9041
303 0.04585814618669408 0.630111246411919 0.92916 0.9133
304 0.37014618459831944 0.6474331755414998 0.93002 0.9081
305 0.17172639246304608 0.625560964363441 0.9288 0.9137
306 0.35571338678572634 

433 0.42970954323860816 0.6998645227833554 0.93048 0.9084
434 0.06740562294877595 0.7016981189277298 0.93016 0.907
435 0.5344805990056007 0.701155923156799 0.92926 0.9041
436 0.09685529299727486 0.6752903771452333 0.93006 0.9122
437 0.06728941499943362 0.7165006744854915 0.9299 0.9002
438 0.08458385761024494 0.6804944963323676 0.9305 0.9144
439 0.5813497907610631 0.6851835102279465 0.92958 0.9087
440 0.7060521513680859 0.7102689430395344 0.9305 0.9048
441 0.2722462986057086 0.6876945931130811 0.93028 0.9083
442 0.11865378463360651 0.6976284393409607 0.92816 0.907
443 0.3295470218392772 0.7032737546622435 0.93146 0.9025
444 0.08441533213976303 0.7056546361747698 0.9311 0.9053
445 1.435902617171278 0.682657142301418 0.92954 0.9072
446 0.32375944546708163 0.698687027585266 0.93056 0.9071
447 0.017073997923668236 0.7108181499196865 0.9296 0.903
448 0.20712503218116632 0.6904744625916776 0.92956 0.9111
449 0.5384198180313431 0.707312893381535 0.93102 0.904
450 0.07242222057417705 0.68633495

577 0.07357985324047646 0.7416257120568991 0.93068 0.9076
578 0.1538437390811686 0.7708742525808475 0.92994 0.8927
579 0.2234848981854397 0.7336973381848473 0.93078 0.9094
580 0.4012142634344483 0.7355451957970397 0.93006 0.907
581 0.8358031363291298 0.7289898544522211 0.93022 0.9099
582 0.8002278064293664 0.7288807338291637 0.92952 0.9107
583 1.1564486701998737 0.7296532896940098 0.93048 0.908
584 0.6322154400511623 0.7303506809517936 0.93046 0.9065
585 0.4033375132762317 0.7404756532259361 0.92984 0.9065
586 0.11992541482742143 0.7262442393463681 0.93048 0.9108
587 0.45095185209428384 0.7683923545410393 0.93088 0.8999
588 0.07158873440809567 0.7457696201744201 0.9306 0.9024
589 0.08251253763630016 0.7312040639855946 0.93044 0.9078
590 0.06683437486755547 0.7444752945028111 0.93054 0.9045
591 0.025892384413369746 0.7393478514743383 0.92966 0.9066
592 0.0076054908419586386 0.7415722615329005 0.92986 0.9092
593 0.14961563363996785 0.7398962986193385 0.92864 0.9088
594 0.8861577388985146

...

8735 nan nan 0.09904 0.098
8736 nan nan 0.09904 0.098
8737 nan nan 0.09904 0.098
8738 nan nan 0.09904 0.098
8739 nan nan 0.09904 0.098
8740 nan nan 0.09904 0.098
8741 nan nan 0.09904 0.098
8742 nan nan 0.09904 0.098
8743 nan nan 0.09904 0.098
8744 nan nan 0.09904 0.098
8745 nan nan 0.09904 0.098
8746 nan nan 0.09904 0.098
8747 nan nan 0.09904 0.098
8748 nan nan 0.09904 0.098
8749 nan nan 0.09904 0.098
8750 nan nan 0.09904 0.098
8751 nan nan 0.09904 0.098
8752 nan nan 0.09904 0.098
8753 nan nan 0.09904 0.098
8754 nan nan 0.09904 0.098
8755 nan nan 0.09904 0.098
8756 nan nan 0.09904 0.098
8757 nan nan 0.09904 0.098
8758 nan nan 0.09904 0.098
8759 nan nan 0.09904 0.098
8760 nan nan 0.09904 0.098
8761 nan nan 0.09904 0.098
8762 nan nan 0.09904 0.098
8763 nan nan 0.09904 0.098
8764 nan nan 0.09904 0.098
8765 nan nan 0.09904 0.098
8766 nan nan 0.09904 0.098
8767 nan nan 0.09904 0.098
8768 nan nan 0.09904 0.098
8769 nan nan 0.09904 0.098
8770 nan nan 0.09904 0.098
8771 nan nan 0.09904 0.098
8

9039 nan nan 0.09904 0.098
9040 nan nan 0.09904 0.098
9041 nan nan 0.09904 0.098
9042 nan nan 0.09904 0.098
9043 nan nan 0.09904 0.098
9044 nan nan 0.09904 0.098
9045 nan nan 0.09904 0.098
9046 nan nan 0.09904 0.098
9047 nan nan 0.09904 0.098
9048 nan nan 0.09904 0.098
9049 nan nan 0.09904 0.098
9050 nan nan 0.09904 0.098
9051 nan nan 0.09904 0.098
9052 nan nan 0.09904 0.098
9053 nan nan 0.09904 0.098
9054 nan nan 0.09904 0.098
9055 nan nan 0.09904 0.098
9056 nan nan 0.09904 0.098
9057 nan nan 0.09904 0.098
9058 nan nan 0.09904 0.098
9059 nan nan 0.09904 0.098
9060 nan nan 0.09904 0.098
9061 nan nan 0.09904 0.098
9062 nan nan 0.09904 0.098
9063 nan nan 0.09904 0.098
9064 nan nan 0.09904 0.098
9065 nan nan 0.09904 0.098
9066 nan nan 0.09904 0.098
9067 nan nan 0.09904 0.098
9068 nan nan 0.09904 0.098
9069 nan nan 0.09904 0.098
9070 nan nan 0.09904 0.098
9071 nan nan 0.09904 0.098
9072 nan nan 0.09904 0.098
9073 nan nan 0.09904 0.098
9074 nan nan 0.09904 0.098
9075 nan nan 0.09904 0.098
9

9343 nan nan 0.09904 0.098
9344 nan nan 0.09904 0.098
9345 nan nan 0.09904 0.098
9346 nan nan 0.09904 0.098
9347 nan nan 0.09904 0.098
9348 nan nan 0.09904 0.098
9349 nan nan 0.09904 0.098
9350 nan nan 0.09904 0.098
9351 nan nan 0.09904 0.098
9352 nan nan 0.09904 0.098
9353 nan nan 0.09904 0.098
9354 nan nan 0.09904 0.098
9355 nan nan 0.09904 0.098
9356 nan nan 0.09904 0.098
9357 nan nan 0.09904 0.098
9358 nan nan 0.09904 0.098
9359 nan nan 0.09904 0.098
9360 nan nan 0.09904 0.098
9361 nan nan 0.09904 0.098
9362 nan nan 0.09904 0.098
9363 nan nan 0.09904 0.098
9364 nan nan 0.09904 0.098
9365 nan nan 0.09904 0.098
9366 nan nan 0.09904 0.098
9367 nan nan 0.09904 0.098
9368 nan nan 0.09904 0.098
9369 nan nan 0.09904 0.098
9370 nan nan 0.09904 0.098
9371 nan nan 0.09904 0.098
9372 nan nan 0.09904 0.098
9373 nan nan 0.09904 0.098
9374 nan nan 0.09904 0.098
9375 nan nan 0.09904 0.098
9376 nan nan 0.09904 0.098
9377 nan nan 0.09904 0.098
9378 nan nan 0.09904 0.098
9379 nan nan 0.09904 0.098
9

9647 nan nan 0.09904 0.098
9648 nan nan 0.09904 0.098
9649 nan nan 0.09904 0.098
9650 nan nan 0.09904 0.098
9651 nan nan 0.09904 0.098
9652 nan nan 0.09904 0.098
9653 nan nan 0.09904 0.098
9654 nan nan 0.09904 0.098
9655 nan nan 0.09904 0.098
9656 nan nan 0.09904 0.098
9657 nan nan 0.09904 0.098
9658 nan nan 0.09904 0.098
9659 nan nan 0.09904 0.098
9660 nan nan 0.09904 0.098
9661 nan nan 0.09904 0.098
9662 nan nan 0.09904 0.098
9663 nan nan 0.09904 0.098
9664 nan nan 0.09904 0.098
9665 nan nan 0.09904 0.098
9666 nan nan 0.09904 0.098
9667 nan nan 0.09904 0.098
9668 nan nan 0.09904 0.098
9669 nan nan 0.09904 0.098
9670 nan nan 0.09904 0.098
9671 nan nan 0.09904 0.098
9672 nan nan 0.09904 0.098
9673 nan nan 0.09904 0.098
9674 nan nan 0.09904 0.098
9675 nan nan 0.09904 0.098
9676 nan nan 0.09904 0.098
9677 nan nan 0.09904 0.098
9678 nan nan 0.09904 0.098
9679 nan nan 0.09904 0.098
9680 nan nan 0.09904 0.098
9681 nan nan 0.09904 0.098
9682 nan nan 0.09904 0.098
9683 nan nan 0.09904 0.098
9

9951 nan nan 0.09904 0.098
9952 nan nan 0.09904 0.098
9953 nan nan 0.09904 0.098
9954 nan nan 0.09904 0.098
9955 nan nan 0.09904 0.098
9956 nan nan 0.09904 0.098
9957 nan nan 0.09904 0.098
9958 nan nan 0.09904 0.098
9959 nan nan 0.09904 0.098
9960 nan nan 0.09904 0.098
9961 nan nan 0.09904 0.098
9962 nan nan 0.09904 0.098
9963 nan nan 0.09904 0.098
9964 nan nan 0.09904 0.098
9965 nan nan 0.09904 0.098
9966 nan nan 0.09904 0.098
9967 nan nan 0.09904 0.098
9968 nan nan 0.09904 0.098
9969 nan nan 0.09904 0.098
9970 nan nan 0.09904 0.098
9971 nan nan 0.09904 0.098
9972 nan nan 0.09904 0.098
9973 nan nan 0.09904 0.098
9974 nan nan 0.09904 0.098
9975 nan nan 0.09904 0.098
9976 nan nan 0.09904 0.098
9977 nan nan 0.09904 0.098
9978 nan nan 0.09904 0.098
9979 nan nan 0.09904 0.098
9980 nan nan 0.09904 0.098
9981 nan nan 0.09904 0.098
9982 nan nan 0.09904 0.098
9983 nan nan 0.09904 0.098
9984 nan nan 0.09904 0.098
9985 nan nan 0.09904 0.098
9986 nan nan 0.09904 0.098
9987 nan nan 0.09904 0.098
9

In [8]:
#define starting value of weights W for gradient descent
init_std_dev = 0.01;
initialW=init_std_dev*np.random.randn(n_features,n_classes)
# create a PyTorch tensor variable for W and b. 
# we will be optimizing over W, b, finding their best value using gradient descent,
#so we need gradient enabled
W = torch.tensor(initialW,requires_grad=True,device=device);
b = torch.zeros((1,n_classes),requires_grad=True,device=device);
# this optimizer will do gradient descent for us
learning_rate = 0.1;
#optimizer = torch.optim.SGD([W,b],lr=learning_rate)
optimizer = torch.optim.Adam([W,b],lr=learning_rate)
n_epochs = 10000;
batch_size = 256;


In [9]:
for i in range(n_epochs):

    
    permutation = np.random.permutation(x_train.shape[0]);
    x_train=x_train[permutation,:]
    y_train=y_train[permutation,:]

    accuracy = 0.0;

    #looping over batch_size
    for j in range(0,n_train,batch_size):
        t_x_train = torch.tensor(x_train[j:j+batch_size,:],requires_grad=False,device=device);
        t_y_train = torch.tensor(y_train[j:j+batch_size,:],requires_grad=False,device=device);

        # clear previous gradient calculations
        optimizer.zero_grad();
        
        # calculate model predictions
        linear_predictions = torch.matmul(t_x_train,W)+b
        activations =  torch.exp(linear_predictions);
    
        #Passing through softmax
        sum_activations = torch.sum(activations,dim=1,keepdim=True)
        normalized_activations = torch.div(activations, sum_activations);
        cross_entropy_loss = torch.sum(torch.multiply(t_y_train,torch.log(normalized_activations)),dim=1 )
        risk = -1.0 * torch.mean(cross_entropy_loss );

        # calculate gradients
        risk.backward();
        
        # take the gradient step
        optimizer.step();
        

        batch_risk=risk.item();

        true_class = np.argmax(t_y_train.detach().cpu().numpy(),axis=1)
        pred_class = np.argmax(normalized_activations.detach().cpu().numpy(),axis=1)
        accuracy += np.count_nonzero(true_class == pred_class);
        
    
    #  turn off gradeints 
        
    with (torch.no_grad()):
      linear_predictions = torch.matmul(t_x_test,W)+b
      activations =  torch.exp(linear_predictions);
    
      #Passing through softmax and calculate test set risk and accuracy
      sum_activations = torch.sum(activations,dim=1,keepdim=True)
      test_normalized_activations = torch.div(activations, sum_activations);
      cross_entropy_loss = torch.sum(torch.multiply(t_y_test,torch.log(test_normalized_activations)),dim=1 )
    
      test_risk = -1.0 * torch.mean(cross_entropy_loss );
    
      test_true_class = np.argmax(t_y_test.detach().cpu().numpy(),axis=1)
      test_pred_class = np.argmax(test_normalized_activations.detach().cpu().numpy(),axis=1)
      test_accuracy = np.count_nonzero(test_true_class == test_pred_class)/test_pred_class.shape[0];
      test_error = 1.0 - test_accuracy;
        
    accuracy = accuracy / float(x_train.shape[0])
    print(i,batch_risk,test_risk.item(),accuracy,test_accuracy)

0 0.3812982863382997 0.6281302810918848 0.87386 0.876
1 0.5464960608073783 0.6371684210051146 0.89516 0.8797
2 0.4551843523021306 0.7790976288122424 0.89516 0.8663
3 0.6620265319641894 0.6584934966737084 0.89698 0.9011
4 0.2836127614791435 0.6920923423250157 0.89768 0.9003
5 0.1404264958258535 0.717731675549995 0.89998 0.895
6 0.9440158135634917 0.6316134879342233 0.90038 0.9044
7 0.46559968985714795 0.7396233458699171 0.90084 0.8922
8 0.4530064897878412 0.6558560093046534 0.9008 0.9042
9 0.3627759727570038 0.7240639367927755 0.9028 0.9043
10 0.3472780806382043 0.8634984586756235 0.90066 0.8826
11 1.0368377181414565 0.7754903368230863 0.90542 0.8882
12 0.7864683042880789 0.8857204304888188 0.90168 0.8862
13 0.5445149923245995 0.8081488432116791 0.90364 0.8835
14 0.5021961848097487 0.814747825983555 0.90368 0.8922
15 0.541282072309939 1.0346010455828623 0.90382 0.8691
16 0.2582341539871906 0.9220655265650024 0.9032 0.8804
17 0.5551615414332722 0.8175533102542512 0.90418 0.8925
18 0.4243

172 nan nan 0.09904 0.098
173 nan nan 0.09904 0.098
174 nan nan 0.09904 0.098
175 nan nan 0.09904 0.098
176 nan nan 0.09904 0.098
177 nan nan 0.09904 0.098
178 nan nan 0.09904 0.098
179 nan nan 0.09904 0.098
180 nan nan 0.09904 0.098
181 nan nan 0.09904 0.098
182 nan nan 0.09904 0.098
183 nan nan 0.09904 0.098
184 nan nan 0.09904 0.098
185 nan nan 0.09904 0.098
186 nan nan 0.09904 0.098
187 nan nan 0.09904 0.098
188 nan nan 0.09904 0.098
189 nan nan 0.09904 0.098
190 nan nan 0.09904 0.098
191 nan nan 0.09904 0.098
192 nan nan 0.09904 0.098
193 nan nan 0.09904 0.098
194 nan nan 0.09904 0.098
195 nan nan 0.09904 0.098
196 nan nan 0.09904 0.098
197 nan nan 0.09904 0.098
198 nan nan 0.09904 0.098
199 nan nan 0.09904 0.098
200 nan nan 0.09904 0.098
201 nan nan 0.09904 0.098
202 nan nan 0.09904 0.098
203 nan nan 0.09904 0.098
204 nan nan 0.09904 0.098
205 nan nan 0.09904 0.098
206 nan nan 0.09904 0.098
207 nan nan 0.09904 0.098
208 nan nan 0.09904 0.098
209 nan nan 0.09904 0.098
210 nan nan 

488 nan nan 0.09904 0.098
489 nan nan 0.09904 0.098
490 nan nan 0.09904 0.098
491 nan nan 0.09904 0.098
492 nan nan 0.09904 0.098
493 nan nan 0.09904 0.098
494 nan nan 0.09904 0.098
495 nan nan 0.09904 0.098
496 nan nan 0.09904 0.098
497 nan nan 0.09904 0.098
498 nan nan 0.09904 0.098
499 nan nan 0.09904 0.098
500 nan nan 0.09904 0.098
501 nan nan 0.09904 0.098
502 nan nan 0.09904 0.098
503 nan nan 0.09904 0.098
504 nan nan 0.09904 0.098
505 nan nan 0.09904 0.098
506 nan nan 0.09904 0.098
507 nan nan 0.09904 0.098
508 nan nan 0.09904 0.098
509 nan nan 0.09904 0.098
510 nan nan 0.09904 0.098
511 nan nan 0.09904 0.098
512 nan nan 0.09904 0.098
513 nan nan 0.09904 0.098
514 nan nan 0.09904 0.098
515 nan nan 0.09904 0.098
516 nan nan 0.09904 0.098
517 nan nan 0.09904 0.098
518 nan nan 0.09904 0.098
519 nan nan 0.09904 0.098
520 nan nan 0.09904 0.098
521 nan nan 0.09904 0.098
522 nan nan 0.09904 0.098
523 nan nan 0.09904 0.098
524 nan nan 0.09904 0.098
525 nan nan 0.09904 0.098
526 nan nan 

804 nan nan 0.09904 0.098
805 nan nan 0.09904 0.098
806 nan nan 0.09904 0.098
807 nan nan 0.09904 0.098
808 nan nan 0.09904 0.098
809 nan nan 0.09904 0.098
810 nan nan 0.09904 0.098
811 nan nan 0.09904 0.098
812 nan nan 0.09904 0.098
813 nan nan 0.09904 0.098
814 nan nan 0.09904 0.098
815 nan nan 0.09904 0.098
816 nan nan 0.09904 0.098
817 nan nan 0.09904 0.098
818 nan nan 0.09904 0.098
819 nan nan 0.09904 0.098
820 nan nan 0.09904 0.098
821 nan nan 0.09904 0.098
822 nan nan 0.09904 0.098
823 nan nan 0.09904 0.098
824 nan nan 0.09904 0.098
825 nan nan 0.09904 0.098
826 nan nan 0.09904 0.098
827 nan nan 0.09904 0.098
828 nan nan 0.09904 0.098
829 nan nan 0.09904 0.098
830 nan nan 0.09904 0.098
831 nan nan 0.09904 0.098
832 nan nan 0.09904 0.098
833 nan nan 0.09904 0.098
834 nan nan 0.09904 0.098
835 nan nan 0.09904 0.098
836 nan nan 0.09904 0.098
837 nan nan 0.09904 0.098
838 nan nan 0.09904 0.098
839 nan nan 0.09904 0.098
840 nan nan 0.09904 0.098
841 nan nan 0.09904 0.098
842 nan nan 

1115 nan nan 0.09904 0.098
1116 nan nan 0.09904 0.098
1117 nan nan 0.09904 0.098
1118 nan nan 0.09904 0.098
1119 nan nan 0.09904 0.098
1120 nan nan 0.09904 0.098
1121 nan nan 0.09904 0.098
1122 nan nan 0.09904 0.098
1123 nan nan 0.09904 0.098
1124 nan nan 0.09904 0.098
1125 nan nan 0.09904 0.098
1126 nan nan 0.09904 0.098
1127 nan nan 0.09904 0.098
1128 nan nan 0.09904 0.098
1129 nan nan 0.09904 0.098
1130 nan nan 0.09904 0.098
1131 nan nan 0.09904 0.098
1132 nan nan 0.09904 0.098
1133 nan nan 0.09904 0.098
1134 nan nan 0.09904 0.098
1135 nan nan 0.09904 0.098
1136 nan nan 0.09904 0.098
1137 nan nan 0.09904 0.098
1138 nan nan 0.09904 0.098
1139 nan nan 0.09904 0.098
1140 nan nan 0.09904 0.098
1141 nan nan 0.09904 0.098
1142 nan nan 0.09904 0.098
1143 nan nan 0.09904 0.098
1144 nan nan 0.09904 0.098
1145 nan nan 0.09904 0.098
1146 nan nan 0.09904 0.098
1147 nan nan 0.09904 0.098
1148 nan nan 0.09904 0.098
1149 nan nan 0.09904 0.098
1150 nan nan 0.09904 0.098
1151 nan nan 0.09904 0.098
1

...

8715 nan nan 0.09904 0.098
8716 nan nan 0.09904 0.098
8717 nan nan 0.09904 0.098
8718 nan nan 0.09904 0.098
8719 nan nan 0.09904 0.098
8720 nan nan 0.09904 0.098
8721 nan nan 0.09904 0.098
8722 nan nan 0.09904 0.098
8723 nan nan 0.09904 0.098
8724 nan nan 0.09904 0.098
8725 nan nan 0.09904 0.098
8726 nan nan 0.09904 0.098
8727 nan nan 0.09904 0.098
8728 nan nan 0.09904 0.098
8729 nan nan 0.09904 0.098
8730 nan nan 0.09904 0.098
8731 nan nan 0.09904 0.098
8732 nan nan 0.09904 0.098
8733 nan nan 0.09904 0.098
8734 nan nan 0.09904 0.098
8735 nan nan 0.09904 0.098
8736 nan nan 0.09904 0.098
8737 nan nan 0.09904 0.098
8738 nan nan 0.09904 0.098
8739 nan nan 0.09904 0.098
8740 nan nan 0.09904 0.098
8741 nan nan 0.09904 0.098
8742 nan nan 0.09904 0.098
8743 nan nan 0.09904 0.098
8744 nan nan 0.09904 0.098
8745 nan nan 0.09904 0.098
8746 nan nan 0.09904 0.098
8747 nan nan 0.09904 0.098
8748 nan nan 0.09904 0.098
8749 nan nan 0.09904 0.098
8750 nan nan 0.09904 0.098
8751 nan nan 0.09904 0.098
8

9019 nan nan 0.09904 0.098
9020 nan nan 0.09904 0.098
9021 nan nan 0.09904 0.098
9022 nan nan 0.09904 0.098
9023 nan nan 0.09904 0.098
9024 nan nan 0.09904 0.098
9025 nan nan 0.09904 0.098
9026 nan nan 0.09904 0.098
9027 nan nan 0.09904 0.098
9028 nan nan 0.09904 0.098
9029 nan nan 0.09904 0.098
9030 nan nan 0.09904 0.098
9031 nan nan 0.09904 0.098
9032 nan nan 0.09904 0.098
9033 nan nan 0.09904 0.098
9034 nan nan 0.09904 0.098
9035 nan nan 0.09904 0.098
9036 nan nan 0.09904 0.098
9037 nan nan 0.09904 0.098
9038 nan nan 0.09904 0.098
9039 nan nan 0.09904 0.098
9040 nan nan 0.09904 0.098
9041 nan nan 0.09904 0.098
9042 nan nan 0.09904 0.098
9043 nan nan 0.09904 0.098
9044 nan nan 0.09904 0.098
9045 nan nan 0.09904 0.098
9046 nan nan 0.09904 0.098
9047 nan nan 0.09904 0.098
9048 nan nan 0.09904 0.098
9049 nan nan 0.09904 0.098
9050 nan nan 0.09904 0.098
9051 nan nan 0.09904 0.098
9052 nan nan 0.09904 0.098
9053 nan nan 0.09904 0.098
9054 nan nan 0.09904 0.098
9055 nan nan 0.09904 0.098
9

9323 nan nan 0.09904 0.098
9324 nan nan 0.09904 0.098
9325 nan nan 0.09904 0.098
9326 nan nan 0.09904 0.098
9327 nan nan 0.09904 0.098
9328 nan nan 0.09904 0.098
9329 nan nan 0.09904 0.098
9330 nan nan 0.09904 0.098
9331 nan nan 0.09904 0.098
9332 nan nan 0.09904 0.098
9333 nan nan 0.09904 0.098
9334 nan nan 0.09904 0.098
9335 nan nan 0.09904 0.098
9336 nan nan 0.09904 0.098
9337 nan nan 0.09904 0.098
9338 nan nan 0.09904 0.098
9339 nan nan 0.09904 0.098
9340 nan nan 0.09904 0.098
9341 nan nan 0.09904 0.098
9342 nan nan 0.09904 0.098
9343 nan nan 0.09904 0.098
9344 nan nan 0.09904 0.098
9345 nan nan 0.09904 0.098
9346 nan nan 0.09904 0.098
9347 nan nan 0.09904 0.098
9348 nan nan 0.09904 0.098
9349 nan nan 0.09904 0.098
9350 nan nan 0.09904 0.098
9351 nan nan 0.09904 0.098
9352 nan nan 0.09904 0.098
9353 nan nan 0.09904 0.098
9354 nan nan 0.09904 0.098
9355 nan nan 0.09904 0.098
9356 nan nan 0.09904 0.098
9357 nan nan 0.09904 0.098
9358 nan nan 0.09904 0.098
9359 nan nan 0.09904 0.098
9

9627 nan nan 0.09904 0.098
9628 nan nan 0.09904 0.098
9629 nan nan 0.09904 0.098
9630 nan nan 0.09904 0.098
9631 nan nan 0.09904 0.098
9632 nan nan 0.09904 0.098
9633 nan nan 0.09904 0.098
9634 nan nan 0.09904 0.098
9635 nan nan 0.09904 0.098
9636 nan nan 0.09904 0.098
9637 nan nan 0.09904 0.098
9638 nan nan 0.09904 0.098
9639 nan nan 0.09904 0.098
9640 nan nan 0.09904 0.098
9641 nan nan 0.09904 0.098
9642 nan nan 0.09904 0.098
9643 nan nan 0.09904 0.098
9644 nan nan 0.09904 0.098
9645 nan nan 0.09904 0.098
9646 nan nan 0.09904 0.098
9647 nan nan 0.09904 0.098
9648 nan nan 0.09904 0.098
9649 nan nan 0.09904 0.098
9650 nan nan 0.09904 0.098
9651 nan nan 0.09904 0.098
9652 nan nan 0.09904 0.098
9653 nan nan 0.09904 0.098
9654 nan nan 0.09904 0.098
9655 nan nan 0.09904 0.098
9656 nan nan 0.09904 0.098
9657 nan nan 0.09904 0.098
9658 nan nan 0.09904 0.098
9659 nan nan 0.09904 0.098
9660 nan nan 0.09904 0.098
9661 nan nan 0.09904 0.098
9662 nan nan 0.09904 0.098
9663 nan nan 0.09904 0.098
9

9931 nan nan 0.09904 0.098
9932 nan nan 0.09904 0.098
9933 nan nan 0.09904 0.098
9934 nan nan 0.09904 0.098
9935 nan nan 0.09904 0.098
9936 nan nan 0.09904 0.098
9937 nan nan 0.09904 0.098
9938 nan nan 0.09904 0.098
9939 nan nan 0.09904 0.098
9940 nan nan 0.09904 0.098
9941 nan nan 0.09904 0.098
9942 nan nan 0.09904 0.098
9943 nan nan 0.09904 0.098
9944 nan nan 0.09904 0.098
9945 nan nan 0.09904 0.098
9946 nan nan 0.09904 0.098
9947 nan nan 0.09904 0.098
9948 nan nan 0.09904 0.098
9949 nan nan 0.09904 0.098
9950 nan nan 0.09904 0.098
9951 nan nan 0.09904 0.098
9952 nan nan 0.09904 0.098
9953 nan nan 0.09904 0.098
9954 nan nan 0.09904 0.098
9955 nan nan 0.09904 0.098
9956 nan nan 0.09904 0.098
9957 nan nan 0.09904 0.098
9958 nan nan 0.09904 0.098
9959 nan nan 0.09904 0.098
9960 nan nan 0.09904 0.098
9961 nan nan 0.09904 0.098
9962 nan nan 0.09904 0.098
9963 nan nan 0.09904 0.098
9964 nan nan 0.09904 0.098
9965 nan nan 0.09904 0.098
9966 nan nan 0.09904 0.098
9967 nan nan 0.09904 0.098
9

In [11]:
#define starting value of weights W for gradient descent
init_std_dev = 0.01;
initialW=init_std_dev*np.random.randn(n_features,n_classes)
# create a PyTorch tensor variable for W and b. 
# we will be optimizing over W, b, finding their best value using gradient descent,
#so we need gradient enabled
W = torch.tensor(initialW,requires_grad=True,device=device);
b = torch.zeros((1,n_classes),requires_grad=True,device=device);
# this optimizer will do gradient descent for us
learning_rate = 0.001;
#optimizer = torch.optim.SGD([W,b],lr=learning_rate)
optimizer = torch.optim.Adam([W,b],lr=learning_rate)
n_epochs = 10000;
batch_size = 32;

In [ ]:
for i in range(n_epochs):

    
    permutation = np.random.permutation(x_train.shape[0]);
    x_train=x_train[permutation,:]
    y_train=y_train[permutation,:]

    accuracy = 0.0;

    #looping over batch_size
    for j in range(0,n_train,batch_size):
        t_x_train = torch.tensor(x_train[j:j+batch_size,:],requires_grad=False,device=device);
        t_y_train = torch.tensor(y_train[j:j+batch_size,:],requires_grad=False,device=device);

        # clear previous gradient calculations
        optimizer.zero_grad();
        
        # calculate model predictions
        linear_predictions = torch.matmul(t_x_train,W)+b
        activations =  torch.exp(linear_predictions);
    
        #Passing through softmax
        sum_activations = torch.sum(activations,dim=1,keepdim=True)
        normalized_activations = torch.div(activations, sum_activations);
        cross_entropy_loss = torch.sum(torch.multiply(t_y_train,torch.log(normalized_activations)),dim=1 )
        risk = -1.0 * torch.mean(cross_entropy_loss );

        # calculate gradients
        risk.backward();
        
        # take the gradient step
        optimizer.step();
        

        batch_risk=risk.item();

        true_class = np.argmax(t_y_train.detach().cpu().numpy(),axis=1)
        pred_class = np.argmax(normalized_activations.detach().cpu().numpy(),axis=1)
        accuracy += np.count_nonzero(true_class == pred_class);
        
 
    #  turn off gradeints 
        
    with (torch.no_grad()):
      linear_predictions = torch.matmul(t_x_test,W)+b
      activations =  torch.exp(linear_predictions);
    
      #Passing through softmax and calculate test set risk and accuracy
      sum_activations = torch.sum(activations,dim=1,keepdim=True)
      test_normalized_activations = torch.div(activations, sum_activations);
      cross_entropy_loss = torch.sum(torch.multiply(t_y_test,torch.log(test_normalized_activations)),dim=1 )
    
      test_risk = -1.0 * torch.mean(cross_entropy_loss );
    
      test_true_class = np.argmax(t_y_test.detach().cpu().numpy(),axis=1)
      test_pred_class = np.argmax(test_normalized_activations.detach().cpu().numpy(),axis=1)
      test_accuracy = np.count_nonzero(test_true_class == test_pred_class)/test_pred_class.shape[0];
      test_error = 1.0 - test_accuracy;
        
    accuracy = accuracy / float(x_train.shape[0])
    print(i,batch_risk,test_risk.item(),accuracy,test_accuracy)

0 0.5010696130422158 0.3146942864149373 0.8783 0.9148
1 0.2637905809169177 0.2856012795278559 0.91394 0.9218
2 0.34771161148991775 0.2779358607882004 0.92112 0.9226
3 0.43190000386168426 0.2739386560165226 0.92238 0.9234
4 0.261925963158252 0.27138124721951973 0.92524 0.926
5 0.05678261416182177 0.27174492472483947 0.92644 0.9247
6 0.5617119106758732 0.2697894468823374 0.92796 0.9251
7 0.1935150287074403 0.26652328860637053 0.9292 0.9256
8 0.07541953691535097 0.2685039908370302 0.92996 0.9268
9 0.11860285072906727 0.270716897179824 0.93032 0.9259
10 0.12812147119039238 0.26952147130281245 0.93126 0.9272
11 0.23585472219434075 0.26824983428454163 0.93188 0.9258
12 0.012437219471338606 0.26938149465737293 0.93262 0.9256
13 0.03662003007834741 0.2700798533622536 0.933 0.927
14 0.16866396751067636 0.27214530170998513 0.93294 0.9264
15 0.7843382429741117 0.2731082876130412 0.93352 0.9258
16 0.3522633554737509 0.2748225048431147 0.9345 0.9248
17 0.13045493420332166 0.27344129350760116 0.9346

145 0.09554427372679743 0.33955453171172023 0.94064 0.9235
146 0.24921156968811264 0.34079500214543956 0.9412 0.9238
147 0.05999677502701348 0.33944759462561214 0.94088 0.9243
148 0.23255377394586257 0.34345850337826306 0.94146 0.9231
149 0.09873554025007084 0.3415594208582125 0.93994 0.9238
150 0.03589966484729536 0.34340943365307414 0.94114 0.9234
151 0.05954565916997343 0.34104959068637103 0.94114 0.9244
152 0.22124333186297956 0.3469857617561982 0.94098 0.9221
153 0.23310374306744286 0.34608407099840577 0.94206 0.9222
154 0.6472242257800531 0.344017358351261 0.94086 0.9211
155 0.22797253719795438 0.3452248088209076 0.94118 0.9235
156 0.0866333274408229 0.3457480629315914 0.94124 0.923
157 0.12126493620316968 0.343986707833897 0.94128 0.9232
158 0.23350145017283022 0.34525928036636266 0.9408 0.9232
159 0.21714683184713254 0.3503973373093776 0.94148 0.92
160 0.10954583830843766 0.34689122022021723 0.94122 0.9223
161 0.32407482174806534 0.34322108986349686 0.9417 0.9235
162 0.21511097

287 0.36501777989111 0.3832910100158022 0.9428 0.9205
288 0.32418737743116016 0.38261720304929564 0.94282 0.9209
289 0.13250399989275696 0.38066777790995004 0.94216 0.9213
290 0.2221077901985156 0.38684718511946814 0.94274 0.9195
291 0.08342535402114509 0.38153801453875413 0.94228 0.921
292 0.5538853335431613 0.3841496832359483 0.94276 0.9213
293 0.36573557229656284 0.3832596293379576 0.94294 0.9206
294 0.05805880937823238 0.38400338679405965 0.9419 0.9209
295 0.12164516085939182 0.3850596263731088 0.94228 0.9207
296 0.1512158596748411 0.38808545226685603 0.9428 0.92
297 0.2040673541780325 0.3894881030277352 0.94242 0.9209
298 0.4900684214409248 0.3855399728963956 0.94272 0.9198
299 0.03326249469081933 0.38834768811542925 0.94246 0.9195
300 0.062327157865676996 0.3849944247308596 0.94252 0.921
301 0.10452673946565677 0.38689928773122384 0.94212 0.922
302 0.18437697432788497 0.3852325813117069 0.9424 0.9221
303 0.0494881703045234 0.38683037822190913 0.94276 0.9232
304 0.0938234808104056

429 0.07066384185227707 0.41512040345436846 0.94352 0.9205
430 0.06972723045334858 0.4117281786542522 0.9437 0.921
431 0.07906040313747027 0.4131425597438005 0.9427 0.9187
432 0.12601887677418558 0.41406855198458026 0.94368 0.9195
433 0.18456266077750078 0.4152351821975383 0.94302 0.92
434 0.6234745914170898 0.41386664906303583 0.94278 0.9214
435 0.33609305450260746 0.4134750484702295 0.9428 0.9207
436 0.6727147374176029 0.41605202066590674 0.94288 0.9197
437 0.05111503347995618 0.4170017978217048 0.9427 0.919
438 0.44281654140731935 0.4210827106766762 0.94252 0.9179
439 0.19254594294312363 0.4183773660060251 0.9433 0.9198
440 0.03165975536031555 0.41528834482223664 0.94312 0.9204
441 0.11785593163300992 0.4167248798656261 0.94314 0.9199
442 0.18031284250439555 0.4175904738029666 0.94268 0.92
443 0.41654087844642207 0.4156115056287921 0.94316 0.921
444 0.11261990369950314 0.4214003271636458 0.94292 0.9171
445 0.44190711623124035 0.4158090257404375 0.94262 0.9202
446 0.05058378122008292

571 0.029229323741506146 0.4382650378971473 0.94392 0.919
572 0.33703671638946603 0.43929242492632736 0.94312 0.9194
573 0.29831835497836584 0.43779517692558095 0.94348 0.9188
574 0.16032546750938034 0.4417869765378391 0.94304 0.9181
575 0.027291466386232914 0.4389582120196872 0.94368 0.9205
576 0.33356648837973213 0.4408728299794666 0.9436 0.9184
577 0.5669508240189427 0.4421663270009574 0.94368 0.9187
578 0.05561941266196073 0.4404788492808584 0.94362 0.9183
579 0.23209282364421668 0.4418464317099637 0.94378 0.9192
580 0.3819291618557462 0.43840410461717794 0.94314 0.919
581 0.30935931815448037 0.4456879824984227 0.9443 0.9191
582 0.08421564900599172 0.4424902550222811 0.94324 0.9193
583 0.06936343109045086 0.44688317573197367 0.94386 0.9154
584 0.07586220887914652 0.44482011548352246 0.94328 0.9191
585 0.34029141246073924 0.44606842090805887 0.94382 0.9189
586 0.11110398012215565 0.44627408512792544 0.94384 0.9175
587 0.0933663075555776 0.4431819371088082 0.94294 0.9182
588 0.084236

...

4255 nan nan 0.09904 0.098
4256 nan nan 0.09904 0.098
4257 nan nan 0.09904 0.098
4258 nan nan 0.09904 0.098
4259 nan nan 0.09904 0.098
4260 nan nan 0.09904 0.098
4261 nan nan 0.09904 0.098
4262 nan nan 0.09904 0.098
4263 nan nan 0.09904 0.098
4264 nan nan 0.09904 0.098
4265 nan nan 0.09904 0.098
4266 nan nan 0.09904 0.098
4267 nan nan 0.09904 0.098
4268 nan nan 0.09904 0.098
4269 nan nan 0.09904 0.098
4270 nan nan 0.09904 0.098
4271 nan nan 0.09904 0.098
4272 nan nan 0.09904 0.098
4273 nan nan 0.09904 0.098
4274 nan nan 0.09904 0.098
4275 nan nan 0.09904 0.098
4276 nan nan 0.09904 0.098
4277 nan nan 0.09904 0.098
4278 nan nan 0.09904 0.098
4279 nan nan 0.09904 0.098
4280 nan nan 0.09904 0.098
4281 nan nan 0.09904 0.098
4282 nan nan 0.09904 0.098
4283 nan nan 0.09904 0.098
4284 nan nan 0.09904 0.098
4285 nan nan 0.09904 0.098
4286 nan nan 0.09904 0.098
4287 nan nan 0.09904 0.098
4288 nan nan 0.09904 0.098
4289 nan nan 0.09904 0.098
4290 nan nan 0.09904 0.098
4291 nan nan 0.09904 0.098
4

4559 nan nan 0.09904 0.098
4560 nan nan 0.09904 0.098
4561 nan nan 0.09904 0.098
4562 nan nan 0.09904 0.098
4563 nan nan 0.09904 0.098
4564 nan nan 0.09904 0.098
4565 nan nan 0.09904 0.098
4566 nan nan 0.09904 0.098
4567 nan nan 0.09904 0.098
4568 nan nan 0.09904 0.098
4569 nan nan 0.09904 0.098
4570 nan nan 0.09904 0.098
4571 nan nan 0.09904 0.098
4572 nan nan 0.09904 0.098
4573 nan nan 0.09904 0.098
4574 nan nan 0.09904 0.098
4575 nan nan 0.09904 0.098
4576 nan nan 0.09904 0.098
4577 nan nan 0.09904 0.098
4578 nan nan 0.09904 0.098
4579 nan nan 0.09904 0.098
4580 nan nan 0.09904 0.098
4581 nan nan 0.09904 0.098
4582 nan nan 0.09904 0.098
4583 nan nan 0.09904 0.098
4584 nan nan 0.09904 0.098
4585 nan nan 0.09904 0.098
4586 nan nan 0.09904 0.098
4587 nan nan 0.09904 0.098
4588 nan nan 0.09904 0.098
4589 nan nan 0.09904 0.098
4590 nan nan 0.09904 0.098
4591 nan nan 0.09904 0.098
4592 nan nan 0.09904 0.098
4593 nan nan 0.09904 0.098
4594 nan nan 0.09904 0.098
4595 nan nan 0.09904 0.098
4

4863 nan nan 0.09904 0.098
4864 nan nan 0.09904 0.098
4865 nan nan 0.09904 0.098
4866 nan nan 0.09904 0.098
4867 nan nan 0.09904 0.098
4868 nan nan 0.09904 0.098
4869 nan nan 0.09904 0.098
4870 nan nan 0.09904 0.098
4871 nan nan 0.09904 0.098
4872 nan nan 0.09904 0.098
4873 nan nan 0.09904 0.098
4874 nan nan 0.09904 0.098
4875 nan nan 0.09904 0.098
4876 nan nan 0.09904 0.098
4877 nan nan 0.09904 0.098
4878 nan nan 0.09904 0.098
4879 nan nan 0.09904 0.098
4880 nan nan 0.09904 0.098
4881 nan nan 0.09904 0.098
4882 nan nan 0.09904 0.098
4883 nan nan 0.09904 0.098
4884 nan nan 0.09904 0.098
4885 nan nan 0.09904 0.098
4886 nan nan 0.09904 0.098
4887 nan nan 0.09904 0.098
4888 nan nan 0.09904 0.098
4889 nan nan 0.09904 0.098
4890 nan nan 0.09904 0.098
4891 nan nan 0.09904 0.098
4892 nan nan 0.09904 0.098
4893 nan nan 0.09904 0.098
4894 nan nan 0.09904 0.098
4895 nan nan 0.09904 0.098
4896 nan nan 0.09904 0.098
4897 nan nan 0.09904 0.098
4898 nan nan 0.09904 0.098
4899 nan nan 0.09904 0.098
4

5167 nan nan 0.09904 0.098
5168 nan nan 0.09904 0.098
5169 nan nan 0.09904 0.098
5170 nan nan 0.09904 0.098
5171 nan nan 0.09904 0.098
5172 nan nan 0.09904 0.098
5173 nan nan 0.09904 0.098
5174 nan nan 0.09904 0.098
5175 nan nan 0.09904 0.098
5176 nan nan 0.09904 0.098
5177 nan nan 0.09904 0.098
5178 nan nan 0.09904 0.098
5179 nan nan 0.09904 0.098
5180 nan nan 0.09904 0.098
5181 nan nan 0.09904 0.098
5182 nan nan 0.09904 0.098
5183 nan nan 0.09904 0.098
5184 nan nan 0.09904 0.098
5185 nan nan 0.09904 0.098
5186 nan nan 0.09904 0.098
5187 nan nan 0.09904 0.098
5188 nan nan 0.09904 0.098
5189 nan nan 0.09904 0.098
5190 nan nan 0.09904 0.098
5191 nan nan 0.09904 0.098
5192 nan nan 0.09904 0.098
5193 nan nan 0.09904 0.098
5194 nan nan 0.09904 0.098
5195 nan nan 0.09904 0.098
5196 nan nan 0.09904 0.098
5197 nan nan 0.09904 0.098
5198 nan nan 0.09904 0.098
5199 nan nan 0.09904 0.098
5200 nan nan 0.09904 0.098
5201 nan nan 0.09904 0.098
5202 nan nan 0.09904 0.098
5203 nan nan 0.09904 0.098
5

5471 nan nan 0.09904 0.098
5472 nan nan 0.09904 0.098
5473 nan nan 0.09904 0.098
5474 nan nan 0.09904 0.098
5475 nan nan 0.09904 0.098
5476 nan nan 0.09904 0.098
5477 nan nan 0.09904 0.098
5478 nan nan 0.09904 0.098
5479 nan nan 0.09904 0.098
5480 nan nan 0.09904 0.098
5481 nan nan 0.09904 0.098
5482 nan nan 0.09904 0.098
5483 nan nan 0.09904 0.098
5484 nan nan 0.09904 0.098
5485 nan nan 0.09904 0.098
5486 nan nan 0.09904 0.098
5487 nan nan 0.09904 0.098
5488 nan nan 0.09904 0.098
5489 nan nan 0.09904 0.098
5490 nan nan 0.09904 0.098
5491 nan nan 0.09904 0.098
5492 nan nan 0.09904 0.098
5493 nan nan 0.09904 0.098
5494 nan nan 0.09904 0.098
5495 nan nan 0.09904 0.098
5496 nan nan 0.09904 0.098
5497 nan nan 0.09904 0.098
5498 nan nan 0.09904 0.098
5499 nan nan 0.09904 0.098
5500 nan nan 0.09904 0.098
5501 nan nan 0.09904 0.098
5502 nan nan 0.09904 0.098
5503 nan nan 0.09904 0.098
5504 nan nan 0.09904 0.098
5505 nan nan 0.09904 0.098
5506 nan nan 0.09904 0.098
5507 nan nan 0.09904 0.098
5